In [1]:
from HurdatReader import *
from PlotPaths import *
import numpy as np
import scipy.stats as st
import pandas as pd
hurdat = trimHurdat(readHurdat('Data/Hurdat.txt'), timeMin = dt.datetime(1900, 1, 1, tzinfo = dt.timezone.utc))

In [103]:
def statistics_before_time(hurdat, t_minus_time, lon_min, lon_max, lat_min, lat_max):
    storm_df = pd.DataFrame()
    for storm in hurdat:
        new_row = positionBeforeArrival(hurdat[storm], t_minus_time, lon_min, lon_max, lat_min, lat_max, category_statistics=True)
        if None in new_row:
            continue
        #print(new_row)
        storm_df = pd.concat([storm_df, new_row], axis = 1)
        
    storm_df = storm_df.T
    
    print(f'{len(storm_df)} storms considered')
    
    status_df = pd.DataFrame()
    
    STORM_STATUSES = ['TS+', 'TS-', 'EX', 'TD', 'HU1', 'HU2', 'HU3', 'HU4', 'HU5', 'LO', 'SD', 'SS', 'DB']
    SKIP_LIST = ['datetime', 'record identifier', 'system status']
    TUPLE_LIST = ['34 knot wind radii max extent (NM)', '50 knot wind radii max extent (NM)', '64 knot wind radii max extent (NM)']
    #max wind radii are in there since the data is all kinds of messed up
    
    for status in STORM_STATUSES:
        status_series = pd.Series(status, dtype=object)
        status_list = storm_df[storm_df['system status'] == status]
        if len(status_list) == 0:
            continue
        for key in status_list:
            if key in SKIP_LIST:
                continue
            elif key in TUPLE_LIST:
                masked_col_temp = []
                for value in status_list[key]:
                    if ~np.any(pd.isnull(value)):
                        masked_col_temp.append(value)
                masked_col = np.ones(np.shape(masked_col_temp))
                for i, row in enumerate(masked_col_temp):
                    masked_col[i] = row
                if len(masked_col)==0:
                    continue
                status_series[key] = {'mean': np.mean(masked_col.T, axis=1), 'std': np.std(masked_col.T, axis=1)}
            else:
                masked_col = np.ma.array(status_list[key], mask=pd.isnull(status_list[key]), dtype=float)
                status_series[key] = {'mean': np.mean(masked_col), 'std': np.std(masked_col), 
                                      'min': np.min(masked_col), 'max': np.max(masked_col),
                                     'percentiles': np.nanpercentile(status_list[key], np.arange(10,100,10))}
        status_df = pd.concat([status_df, status_series], axis = 1)
        status_df.columns = status_df.iloc[0]
        final_df = status_df.T
        del final_df[0]
    
    return final_df

In [105]:
observation_list =  statistics_before_time(hurdat, 24, lon_min=-73, lon_max=-69.5, lat_min=41, lat_max=43)
#observation_list.rename(observation_list[0])
observation_list

51 storms considered


/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
/home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1560: RuntimeWarning: All-NaN slice encountered
  r

,lat,lon,Average forward speed (mph),Max sustained wind (knots),Minimum Pressure (millibars),34 knot wind radii max extent (NM),50 knot wind radii max extent (NM),64 knot wind radii max extent (NM),max wind radius (NM)
0,,,,,,,,,
TS+,"{'mean': 34.98057480115053, 'std': 1.298110107...","{'mean': -74.99771056867164, 'std': 3.92281856...","{'mean': 18.35937056533145, 'std': 4.846177962...","{'mean': 57.31060606060607, 'std': 18.77848311...","{'mean': 974.5, 'std': 18.5, 'min': 956.0, 'ma...","{'mean': [50.0, 132.5, 50.0, 25.0], 'std': [50...","{'mean': [30.0, 40.0, 0.0, 0.0], 'std': [30.0,...","{'mean': [0.0, 20.0, 0.0, 0.0], 'std': [0.0, 2...","{'mean': 60.0, 'std': 30.0, 'min': 30.0, 'max'..."
TS-,"{'mean': 36.066787037594686, 'std': 2.05538744...","{'mean': -75.4525634703807, 'std': 2.594722618...","{'mean': 18.911809229067476, 'std': 7.58693133...","{'mean': 55.714285714285715, 'std': 18.4058553...","{'mean': 992.6, 'std': 15.134067529914091, 'mi...","{'mean': [85.0, 70.0, 50.0, 45.0], 'std': [35....","{'mean': [32.5, 32.5, 10.0, 10.0], 'std': [7.5...","{'mean': [0.0, 0.0, 0.0, 0.0], 'std': [0.0, 0....","{'mean': --, 'std': --, 'min': --, 'max': --, ..."
EX,"{'mean': 35.699999999999996, 'std': 3.18812902...","{'mean': -77.91666666666667, 'std': 3.37819807...","{'mean': 22.094629411417586, 'std': 10.2016627...","{'mean': 41.666666666666664, 'std': 13.6676828...","{'mean': 999.5, 'std': 9.647970425604202, 'min...","{'mean': [62.5, 75.625, 33.125, 5.0], 'std': [...","{'mean': [13.75, 16.875, 2.5, 0.0], 'std': [23...","{'mean': [3.125, 5.0, 0.0, 0.0], 'std': [8.267...","{'mean': 200.0, 'std': 100.0, 'min': 100.0, 'm..."
TD,"{'mean': 37.95, 'std': 0.5500000000000007, 'mi...","{'mean': -76.8, 'std': 2.799999999999997, 'min...","{'mean': 15.632615758248459, 'std': 6.43692690...","{'mean': 27.5, 'std': 12.5, 'min': 15.0, 'max'...","{'mean': 1007.5, 'std': 3.5, 'min': 1004.0, 'm...",NaN,NaN,NaN,"{'mean': --, 'std': --, 'min': --, 'max': --, ..."
HU1,"{'mean': 31.5683293202947, 'std': 0.1683293202...","{'mean': -77.78783487052428, 'std': 1.58783487...","{'mean': 24.514169367587193, 'std': 3.75079306...","{'mean': 84.16666666666666, 'std': 0.833333333...","{'mean': 944.0, 'std': 0.0, 'min': 944.0, 'max...",NaN,NaN,NaN,"{'mean': --, 'std': --, 'min': --, 'max': --, ..."
HU2,"{'mean': 32.5779888057933, 'std': 0.2727860942...","{'mean': -75.78612101760018, 'std': 0.27558797...","{'mean': 18.579038056829926, 'std': 1.55095975...","{'mean': 102.5925925925926, 'std': 12.90463065...","{'mean': --, 'std': --, 'min': --, 'max': --, ...",NaN,NaN,NaN,"{'mean': --, 'std': --, 'min': --, 'max': --, ..."
HU3,"{'mean': 30.750270622491506, 'std': 1.85019019...","{'mean': -76.12550467818653, 'std': 1.27619232...","{'mean': 19.007656663050177, 'std': 5.33332721...","{'mean': 108.75, 'std': 16.25, 'min': 92.5, 'm...","{'mean': --, 'std': --, 'min': --, 'max': --, ...",NaN,NaN,NaN,"{'mean': --, 'std': --, 'min': --, 'max': --, ..."
